In [76]:
type Node = { 
    x: int; 
    y: int 
}
module Node =
    let Empty = { x = 0; y = 0 }
    let up node =
        { node with y = node.y + 1 } 
    let down node =
        { node with y = node.y - 1 } 
    let left node =
        { node with x = node.x - 1 }
    let right node =
        { node with x = node.x + 1 }

module Tail =

    let abs (it:int) = Math.Abs(it)

    let follow head tail =
        let xoffset = head.x - tail.x
        let yoffset = head.y - tail.y
        
        let adjustX =
            if xoffset > 0 then Node.right else Node.left

        let adjustY =
            if yoffset > 0 then Node.up else Node.down

        match Math.Abs(xoffset), Math.Abs(yoffset) with
        | x, y when x <= 1 && y <= 1 ->
            tail
        | x, y when x = 2 && y = 0 ->
            tail |> adjustX
        | x, y when x = 0 && y = 2 ->
            tail |> adjustY
        | _ -> tail |> adjustX |> adjustY

type Rope = { 
    head: Node; tail: Node 
} with 
    static member Empty = { head = Node.Empty; tail = Node.Empty }
module Rope =
    let move (direction:Node->Node) (rope:Rope) =
        let newHead = rope.head |> direction
        let newTail = rope.tail |> Tail.follow newHead
        { head = newHead; tail = newTail }

let unpack (line:string) =
    match line.Split(" ") with 
    | [| dir; steps |] -> seq { for _ in 1..(steps |> int) do yield dir }
    | _ -> Seq.empty

let parser (move:string) (rope:Rope) =
    match move with
    | "U" -> rope |> Rope.move Node.up 
    | "D" -> rope |> Rope.move Node.down
    | "L" -> rope |> Rope.move Node.left 
    | "R" -> rope |> Rope.move Node.right
    | _ -> rope

let folder (rope:Rope) (line:string) =
    let newRope = rope |> parser line
    (newRope), newRope

let ResolutionFolder = __SOURCE_DIRECTORY__
let lines = File.ReadLines(ResolutionFolder + "/testcase9.txt")

lines
|> Seq.map unpack
|> Seq.concat
|> Seq.mapFold folder Rope.Empty
|> fst
|> Seq.map(fun it -> it.tail)
|> Set.ofSeq

// [ "L"; "L"; "L" ]
// |> Seq.mapFold folder { head = { x = 4; y = 4 }; tail = { x = 4; y = 3} }
// |> fst

orthogonal

orthogonal

orthogonal

orthogonal

orthogonal

...

orthogonal

orthogonal

orthogonal

orthogonal

index,x,y
0,0,0
1,1,0
2,2,0
3,3,0
4,3,0
5,4,1
6,4,2
7,4,3
8,4,3
9,3,4
